<a href="https://colab.research.google.com/github/evsemenova/Python-for-Analysts/blob/HW_6/%D0%A3%D1%80%D0%BE%D0%BA_6_%D0%9B%D0%B8%D0%BD%D0%B5%D0%B9%D0%BD%D0%B0%D1%8F_%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0_%D1%81_%D0%B1%D0%B8%D0%B1%D0%BB%D0%B8%D0%BE%D1%82%D0%B5%D0%BA%D0%BE%D0%B9_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Руководство решает внедрить фичу “С этим товаром покупают” в Интернет-
магазине. Вам предлагается протестировать фичу на одном из товаров. Для
тестирования фичи вам исходя из истории покупок в интернет-магазине нужно
определить ТОП-10 наиболее близких товаров к исходному.
1. Используйте датасет с практики текущего урока.
2. Создайте матрицу item-customer (по срокам - товары, по столбцам - покупатели)
3. Проведите оценку мер близости товаров, получив матрицу
item_item_sim_matrix со значениями косинусов между векторами товаров.
4. Отберите ТОП-10 похожих товаров по StockCode.
5. Выведите список ТОП-10 похожих товаров с названиями (Description) на экран.
Исходный товар - StockCode: 23166 Description: MEDIUM CERAMIC TOP STORAGE JAR

In [28]:
# Импортируем нужные библиотеки
import pandas as pd
import numpy as np

In [29]:
#Загрузим датасет
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [30]:
# Смотрим размеры датасета
df.shape

(541909, 8)

In [31]:
# Чистим датасет: удаляем записи с нулевыми покупками
df = df.loc[df['Quantity'] > 0]

In [32]:
# Чистим датасет: удаляем записи с пропусками в customer_id
df = df.dropna(subset=['CustomerID'])

In [33]:
# Проверим датасет после чистки
df.shape

(397924, 8)

In [34]:
# Агрегируем данные в сводную таблицу "товар - покупатель")
item_customer_matrix = df.pivot_table(index='StockCode', columns='CustomerID', values='Quantity', aggfunc='sum')
item_customer_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10080,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Смотрим на размеры таблицы
item_customer_matrix.shape

(3665, 4339)

In [36]:
# Сравниваем кол-во строк с кол-вом уникальных покупателей в исходном датафрейме
df['CustomerID'].nunique()

4339

In [37]:
# Сравниваем кол-во столбцов с кол-вом уникальных товаров в исходном датафрейме
df['StockCode'].nunique()

3665

In [38]:
# # Заменяем кол-во покупок на событие: 1- факт покупки, 0 - отсутствие покупки
item_customer_matrix = item_customer_matrix.applymap(lambda x: 1 if x > 0 else 0)
item_customer_matrix.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,...,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10080,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10125,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10133,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


3. Проведем оценку мер близости товаров, получив матрицу
item_item_sim_matrix со значениями косинусов между векторами товаров.

In [39]:
# Преобразуем исходную матрицу "товар - покупатель" в массив numpy
m = item_customer_matrix.values
m

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [40]:
# Импортируем библиотеку для создания разреженной матрицы
from scipy.sparse import csr_matrix

In [41]:
# Задаем функцию расчета косинуса между векторами-строками товаров 
def cos_sim(array):
  csr_array = csr_matrix(array)
  numerator = np.dot(csr_array,csr_array.T).toarray()
  denomenator = np.linalg.norm(array, axis = 1, keepdims = True)*np.linalg.norm(array.T, axis = 0, keepdims = True)
  return numerator / denomenator

In [42]:
# Вычисляем косинусную меру сходства между векторами-строками товаров 
item_item_sim_matrix = cos_sim(m)
item_item_sim_matrix

array([[1.        , 0.        , 0.09486833, ..., 0.06691496, 0.        ,
        0.07821652],
       [0.        , 1.        , 0.        , ..., 0.01618174, 0.        ,
        0.        ],
       [0.09486833, 0.        , 1.        , ..., 0.07053456, 0.        ,
        0.01099299],
       ...,
       [0.06691496, 0.01618174, 0.07053456, ..., 1.        , 0.        ,
        0.0775386 ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.07821652, 0.        , 0.01099299, ..., 0.0775386 , 0.        ,
        1.        ]])

In [43]:
# Добавляем название столбцов и индексов в матрицу мер близости между векторами-строками товаров
item_item_sim_matrix = pd.DataFrame(item_item_sim_matrix)

item_item_sim_matrix.columns = item_customer_matrix.index

item_item_sim_matrix['StockCode'] = item_customer_matrix.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')

item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,...,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,...,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,...,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


4. Отберем ТОП-10 похожих товаров по StockCode.

In [44]:
top_10 = list(item_item_sim_matrix.loc[23166].sort_values(ascending=False).iloc[:11].index)
top_10

[23166, 23165, 23167, 22993, 23307, 22722, 22720, 22666, 23243, 22961, 23306]

5. Выведите список ТОП-10 похожих товаров с названиями (Description) на экран.

In [45]:
df.loc[df['StockCode'].isin(top_10), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode').loc[top_10]

,Description
StockCode,
23166,MEDIUM CERAMIC TOP STORAGE JAR
23165,LARGE CERAMIC TOP STORAGE JAR
23167,SMALL CERAMIC TOP STORAGE JAR
22993,SET OF 4 PANTRY JELLY MOULDS
23307,SET OF 60 PANTRY DESIGN CAKE CASES
22722,SET OF 6 SPICE TINS PANTRY DESIGN
22720,SET OF 3 CAKE TINS PANTRY DESIGN
22666,RECIPE BOX PANTRY YELLOW DESIGN
23243,SET OF TEA COFFEE SUGAR TINS PANTRY
